In [30]:
!pip install pandas
!pip install numpy
!pip install seaborn
!pip install matplotlib
!pip install sklearn
!pip install librosa
!pip install IPython
!pip install warnings
!pip install os


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement warnings (from versions: none)

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for warnings
ERROR: Could not find a version that satisfies the requirement os (from versions: none)

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for os


In [31]:
import librosa
import numpy as np
import os
import pandas as pd
from typing import Dict, List, Tuple

In [32]:
# Zdefiniujemy wszystkie cechy, które będziemy obliczać
FEATURE_KEYS: List[str] = [
    'tempo',
    'mean_zcr',
    'mean_rms',
    'mean_spectral_centroid',
    'mean_spectral_bandwidth',
    'mean_spectral_rolloff',
] + [f'mfcc_{i}' for i in range(1, 21)] # Dodajemy 20 współczynników MFCC

def extract_features(file_path: str, n_mfcc: int = 20) -> Dict[str, float] | None:
    """Ekstrahuje rozszerzony zestaw cech z pliku audio."""
    try:
        # 1. Ładowanie pliku
        y, sr = librosa.load(file_path, mono=True, duration=30)
    except Exception as e:
        # print(f"Nie można załadować pliku {file_path}: {e}")
        return None

    # 2. Obliczenia cech
    tempo, _ = librosa.beat.beat_track(y=y, sr=sr)

    # Agregacja średnich dla cech w dziedzinie czasu/widma
    zcr = librosa.feature.zero_crossing_rate(y).mean()
    rms = librosa.feature.rms(y=y).mean()
    cent = librosa.feature.spectral_centroid(y=y, sr=sr).mean()
    bw = librosa.feature.spectral_bandwidth(y=y, sr=sr).mean()
    roll = librosa.feature.spectral_rolloff(y=y, sr=sr).mean()

    # MFCC
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)

    features = {
        'tempo': tempo,
        'mean_zcr': zcr,
        'mean_rms': rms,
        'mean_spectral_centroid': cent,
        'mean_spectral_bandwidth': bw,
        'mean_spectral_rolloff': roll
    }

    # Dodanie 20 współczynników MFCC
    for i in range(n_mfcc):
        features[f'mfcc_{i+1}'] = mfccs[i].mean()

    return features


def calculate_all_stats(data_path: str, genres: List[str]) -> Tuple[pd.DataFrame, Dict[str, Dict[str, float]], Dict[str, np.ndarray]]:
    """
    Przetwarza cały dataset, oblicza statystyki globalne i wektory idealne gatunków.
    """

    all_data = []

    # A. Ekstrakcja wszystkich cech i zbieranie danych
    print("Rozpoczynam ekstrakcję cech z całego datasetu (może zająć kilka minut)...")
    for genre in genres:
        genre_path = os.path.join(data_path, genre)
        if not os.path.isdir(genre_path):
            continue

        for filename in os.listdir(genre_path):
            if filename.endswith('.au') or filename.endswith('.wav'):
                file_path = os.path.join(genre_path, filename)
                features = extract_features(file_path)

                if features:
                    features['genre'] = genre
                    all_data.append(features)

    df = pd.DataFrame(all_data)
    print(f"Ekstrakcja zakończona. Przetworzono {len(df)} utworów.")

    # B. Obliczanie statystyk globalnych (do normalizacji)
    global_mean = df[FEATURE_KEYS].mean()
    global_std = df[FEATURE_KEYS].std()

    global_stats = {
        key: {'mu': global_mean[key], 'sigma': global_std[key]}
        for key in FEATURE_KEYS
    }

    # C. Obliczanie Wektorów Idealnych Gatunków (średnia dla każdego gatunku)
    # Wektory te będą użyte jako referencyjne "cele" dla klasyfikacji dystansowej.
    genre_profiles_raw = df.groupby('genre')[FEATURE_KEYS].mean()

    # D. Normalizacja Wektorów Idealnych na Z-Score (względem statystyk globalnych)
    target_profiles: Dict[str, np.ndarray] = {}
    for genre in genre_profiles_raw.index:
        raw_vector = genre_profiles_raw.loc[genre]
        normalized_vector = []
        for key in FEATURE_KEYS:
            mu = global_stats[key]['mu']
            sigma = global_stats[key]['sigma']
            # Wzór Z-Score: (X - mu) / sigma
            normalized_value = (raw_vector[key] - mu) / sigma
            normalized_vector.append(normalized_value)

        target_profiles[genre] = np.array(normalized_vector)

    return df, global_stats, target_profiles

In [33]:
# Te funkcje muszą przyjmować słowniki statystyk i profili jako argumenty!

def normalize_features(features: Dict[str, float], stats: Dict[str, Dict[str, float]]) -> np.ndarray:
    """Normalizuje cechy na Z-Score używając dynamicznie obliczonych statystyk."""
    normalized_values = []
    for key in FEATURE_KEYS:
        value = features[key]
        mu = stats[key]['mu']
        sigma = stats[key]['sigma']
        # Wzór Z-Score: (X - mu) / sigma
        normalized_values.append((value - mu) / sigma)

    return np.array(normalized_values)


def classify_by_distance(input_features: Dict[str, float], stats: Dict[str, Dict[str, float]], profiles: Dict[str, np.ndarray]) -> Tuple[str, float]:
    """
    Klasyfikuje utwór, obliczając najmniejszą odległość Euklidesową (obliczeniowo).
    """

    # 1. Normalizacja cech wejściowych
    input_vector = normalize_features(input_features, stats)

    distances = {}

    # 2. Obliczanie odległości
    for genre, target_vector in profiles.items():
        # Obliczenie Odległości Euklidesowej (norma wektora różnicy)
        distance = np.linalg.norm(input_vector - target_vector)
        distances[genre] = distance

    # 3. Wybór gatunku o najmniejszej odległości
    predicted_genre = min(distances, key=distances.get)
    min_distance = distances[predicted_genre]

    return predicted_genre, min_distance

In [34]:
# --- KONFIGURACJA I URUCHOMIENIE ---

# !!! WAŻNE: ZMIEŃ TĘ ŚCIEŻKĘ NA WŁAŚCIWĄ !!!
GTZAN_DATA_PATH = './lib/gtzan-dataset-music-genre-classification/Data/genres_original'
# Dostępne gatunki w GTZAN
ALL_GENRES = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']

# 1. FAZA TRENINGOWA (Obliczenie Statystyk i Profili)
# W tej fazie Python sam liczy wszystkie stałe.
try:
    _, global_stats, target_profiles = calculate_all_stats(GTZAN_DATA_PATH, ALL_GENRES)

except FileNotFoundError:
    print("\n[BŁĄD] Upewnij się, że ścieżka GTZAN_DATA_PATH jest poprawna i zawiera folder 'genres_original' z podfolderami gatunków.")
    exit()

# Weryfikacja: Zobaczmy, jak wygląda znormalizowany profil Jazzu (obliczony!)
print("\n--- Obliczone Wektory Idealne (Fragment) ---")
print(f"Ilość cech użytych w obliczeniach: {len(FEATURE_KEYS)}")
print(f"Profil 'Jazz' (pierwsze 5 znormalizowanych cech):\n{target_profiles['jazz'][:5]}")
print(f"Profil 'Metal' (pierwsze 5 znormalizowanych cech):\n{target_profiles['metal'][:5]}")
print("-------------------------------------------\n")


# 2. FAZA TESTOWA (Klasyfikacja Utworów)

test_results = []
test_genres = ['jazz', 'metal', 'classical'] # Testujemy tylko wybrane dla przejrzystości

print("Rozpoczynam test klasyfikacji...")
for genre in test_genres:
    genre_path = os.path.join(GTZAN_DATA_PATH, genre)

    # Testujemy tylko kilka pierwszych plików dla każdego gatunku
    for i, filename in enumerate(os.listdir(genre_path)):
        if i >= 5: # Ogranicz do 5 plików na gatunek
            break

        if filename.endswith('.au') or filename.endswith('.wav'):
            file_path = os.path.join(genre_path, filename)

            # A. Ekstrakcja cech z utworu testowego
            features = extract_features(file_path)
            if features is None:
                continue

            # B. Klasyfikacja na podstawie obliczeń dystansowych
            predicted_genre, distance = classify_by_distance(features, global_stats, target_profiles)

            test_results.append({
                'true_genre': genre,
                'predicted_genre': predicted_genre,
                'distance': distance
            })

            print(f"[{genre:10}] -> {predicted_genre:10} | Odległość: {distance:.2f}")

# Podsumowanie
test_df = pd.DataFrame(test_results)
test_accuracy = (test_df['true_genre'] == test_df['predicted_genre']).mean()
print("\n" + "="*50)
print(f"Skuteczność testu ({len(test_df)} plików): {test_accuracy*100:.2f}%")
print("="*50)

Rozpoczynam ekstrakcję cech z całego datasetu (może zająć kilka minut)...
Ekstrakcja zakończona. Przetworzono 999 utworów.


TypeError: Could not convert [array([119263.1622445])] to numeric